## CAP/GBR step 1

In [9]:
show_plots=False

In [2]:
import numpy as np
from scipy.linalg import eig, eigh, eigvals, eigvalsh
import matplotlib.pyplot as plt
#import matplotlib
#matplotlib.use('Qt5Agg')
%matplotlib qt5
import pandas as pd

In [3]:
import sys
sys.path.append('../../Python_libs')
from GTO_basis import GBR
from jolanta import Jolanta_3D

In [4]:
amu_to_au=1822.888486192
au2cm=219474.63068
au2eV=27.211386027
Angs2Bohr=1.8897259886

In [5]:
#
# Jolanata parameters a, b, c:
#
#   CS-DVR:   
#      bound state:  -7.17051 eV
#      resonance (3.1729556 - 0.16085j) eV
#
jparam=(0.028, 1.0, 0.028)

* Create a valence set $[\alpha_0, \alpha_0/s, \alpha_0/s^2, ..., \alpha_N]$ and set the CAP radius $r_c$
* Diagonalize **H** to compare with $E_0^{DVR}$
* Add diffuse functions $[\alpha_N/s_{df}, ...]$
* Diagonalize **H** again

In [49]:
sets=['GTO_unc', 'GTO_DZ', 'GTO_TZ']
bas = sets[0]
nval=10
a0=17.0
s=2
ndf=4
sdf=1.5
if bas == 'GTO_unc':
    contract = (0,0)
elif bas == 'GTO_DZ':
    contract = (1,1)  # one contracted, one uncontracted function
elif bas == 'GTO_TZ':
    contract = (1,2)  # one contracted, two uncontracted function
else:
    print('No such basis.')

rc=7 # start for the CAP
rc=5
fname = 'Traj_' + bas + '_' + str(rc) + '.csv'
print(fname)

Traj_GTO_unc_5.csv


### Valence set
Compare the bound state with DVR: $E_0 = -7.17051$ eV

In [50]:
alpha_val=[a0]
for i in range(nval-1):
    alpha_val.append(alpha_val[-1]/s)
Val = GBR(alpha_val, jparam, contract=contract, diffuse=(0,0))
S, T, V = Val.STV()
Es, cs = eigh(T+V, b=S)
print(f'E0 = {Es[0]*au2eV:.6f}   Emax = {Es[-1]*au2eV:.6f}')
#Val.print_exp()

if show_plots:
    scale=10
    xmax=15
    xs=np.linspace(0.1,xmax,200)
    Vs=Jolanta_3D(xs, jparam)
    plt.cla()
    plt.plot(xs,Vs*au2eV, '-', color="blue")
    for i in range(len(Es)):
        ys=Val.eval_vector(cs[:,i], xs)
        plt.plot(xs,scale*ys**2+Es[i]*au2eV, '-')
    plt.ylim(-8,10)
    plt.show()

E0 = -7.170439   Emax = 2419.320761


### Extend the basis by a diffuse set

In [51]:
Bas = GBR(alpha_val, jparam, contract=contract, diffuse=(ndf,sdf))
S, T, V = Bas.STV()
Es, cs = eigh(T+V, b=S)
nEs = len(Es)
print(f'E0 = {Es[0]*au2eV:.6f}   Emax = {Es[-1]*au2eV:.6f}')

if show_plots:
    Emax=10 # eV
    plt.cla()
    plt.plot(xs,Vs*au2eV, '-', color="blue")
    for i, E in enumerate(Es):
        ys=Bas.eval_vector(cs[:,i], xs)
        plt.plot(xs,scale*ys**2+E*au2eV, '-')
        if E*au2eV > Emax:
            break

    plt.ylim(-10,Emax+1)
    plt.show()

E0 = -7.170447   Emax = 2421.905980


## CAP

In [52]:
W = Bas.Wcap(rc)
np.diag(W)
# Testing library: nval=10, a0=17.0, s=2, ndf=4, sdf=1.5
#array([0.00000000e+000,  0.00000000e+000, -5.90256357e-182,
#      -2.30877456e-091, -5.43012033e-046, -3.13170550e-023,
#       1.99268280e-011,  1.39385969e-005,  1.55622760e-002,
#       7.33295233e-001,  3.21481773e+000,  1.00955294e+001,
#       2.54624133e+001,  5.55012418e+001])

array([ 0.00000000e+000, -6.06589018e-186, -1.66336265e-093,
       -3.27559837e-047, -5.46638222e-024,  5.90510417e-012,
        5.37825528e-006,  6.84173269e-003,  3.43575779e-001,
        3.59316286e+000,  9.66794702e+000,  2.20063339e+001,
        4.47519311e+001,  8.42063227e+001])

### $\eta$-run

We are looking for $min\vert \frac{dE}{d\ln\eta}\vert$, so use a $\log\eta$ scale.

In [53]:
n_keep=nEs
log_eta_min=-5
log_eta_max=0
n_eta=25*(log_eta_max - log_eta_min)+1
etas=np.logspace(log_eta_min, log_eta_max, num=n_eta)

erdata = np.zeros((n_eta,n_keep), complex)  # array used to collect all theta-run data

for i_eta in range(n_eta):
    eta=etas[i_eta]
    H_eta = T + V - 1j*eta*W
    energies = eigvals(H_eta, b=S)
    energies.sort()
    erdata[i_eta,:] = energies[0:n_keep]
    print(i_eta+1, end=" ")
    if (i_eta+1)%10==0:
        print()

erdata *= au2eV

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 
51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 
71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 
91 92 93 94 95 96 97 98 99 100 
101 102 103 104 105 106 107 108 109 110 
111 112 113 114 115 116 117 118 119 120 
121 122 123 124 125 126 

<br> Raw $\eta$ trajectories

In [54]:
plt.cla()
for i in range(0, n_keep):
    plt.plot(erdata[:,i].real,  erdata[:,i].imag, 'o')
plt.xlim(0,6)
plt.ylim(-0.5,0)
plt.show()

<br> Get the resonance trajectory by naive nearest follow

In [55]:
follow=3.3
es=np.zeros(n_eta,complex)

for j in range(0,n_eta):
    i = np.argmin(abs(erdata[j,:]-follow))
    es[j] = erdata[j,i]
    follow = es[j]
plt.cla()
plt.plot(es.real, es.imag, 'o-')
plt.show()

In [56]:
df = pd.DataFrame({"eta": etas, "ReE":es.real, "ImE":es.imag})
df.to_csv(fname, index=False)
print('Wrote',fname)

Wrote Traj_GTO_unc_5.csv


### Analyze with traj_analysis.ipynb